# SIADS 593 WN26 Milestone I: Sustainable Water Quality Notebook

**Authors:**

Sungmin Kim  
Randy Best  
Kyle Rodriguez

#### **Introduction**

Welcome to the Exploratory Data Analysis of water quality across various river locations in South Africa! As a key factor for environmental sustainability, we would like to use data to identify and emphasize the key factors that significantly influence water quality. As a team we will take advantage of the valuable opportunity to apply data exploration techniques to real-world environmental data and contribute to advancing water quality monitoring.

**Primary Dataset**

With a dataset containing three target water quality parameters **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus** collected between 2011 and 2015 from approximately 200 river locations across South Africa, our goal is to find insight on which environmental conditions most affect these 3 metrics. Each data point includes the geographic coordinates (latitude and longitude) of the sampling site, the date of collection, and the corresponding water quality measurements. Features of the dataset also include four spectral bands—**SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1)—were utilized from Landsat, along with derived spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). In addition, the **PET** (Potential Evapotranspiration) variable was incorporated from the **TerraClimate** dataset to account for climatic influences on water quality.

- **SWIR22** – Sensitive to surface moisture and turbidity variations in water bodies.  
- **NIR** – Helps in identifying vegetation and suspended matter in water.  
- **Green** – Useful for detecting water color and surface reflectance changes.  
- **SWIR16** – Provides information on surface dryness and sediment concentration.  
- **NDMI** – Derived from NIR and SWIR16, indicates moisture and vegetation–water interaction.  
- **MNDWI** – Derived from Green and SWIR22, effective for distinguishing open water areas and reducing built-up noise.  
- **PET** – Extracted from the TerraClimate dataset, represents potential evapotranspiration influencing hydrological and water quality dynamics.

The dataset spans a five-year period from 2011 to 2015. Using API-based data extraction methods, both Landsat and TerraClimate features were retrieved directly from the <a href="https://planetarycomputer.microsoft.com/">Microsoft Planetary Computer</a> portal. These combined spectral, index-based, and climatic features were used as predictors in a regression model to estimate three key water quality parameters: Total Alkalinity (TA), Electrical Conductance (EC), and Dissolved Reactive Phosphorus (DRP).

**Secondary Datasets**

Our secondary datasets comprise gridded population estimation from <a href="https://hub.worldpop.org/project/categories?id=18">WorldPop :: Population Density</a> and a <a href="https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa">List of rivers of South Africa</a> from Wikipedia. The population density data includes geographical coordinates and the number of people per square kilometre based on country totals adjusted to match the corresponding official United Nations population estimates that have been prepared by the Population Division of the Department of Economic and Social Affairs of the United Nations Secretariat (<a href="https://population.un.org/wpp/">2019 Revision of World Population Prospects</a>). The list of rivers dataset includes river name, province and location, source location and mouth/junction at their location name and geographical coordinates.

- Gridded population estimates are particularly useful as they provide decision-makers and data users with the flexibility to aggregate population estimates into different spatial units in existing enumeration areas or custom areas. Estimated population density per grid-cell. The projection is Geographic Coordinate System, WGS84. 
- The water sources of each quality sample in our original data are linked to geographical coordinates. So, we will use a simple list of rivers to reference each sample to its nominal water source.
 

 #### **About the Notebook**

In this notebook, we **load previously extracted data** from CSV files generated in a separate extraction notebook. This approach ensures a smoother and faster workflow, allowing us to focus on data analysis without waiting for time-consuming data retrieval.

## Load In Dependencies

To run this demonstration notebook, you will need to have the following packages imported below installed. This may take some time.  

```%pip install numpy pandas geopandas tqdm requests lxml matplotlib plotly seaborn nbformat openpyxl watermark```  
```%pip install plotly[express,jupyter]```  
```%pip install scipy scikit-learn```

In [1]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.neighbors import BallTree
from scipy.spatial import cKDTree

# useful tools
from datetime import date
from tqdm import tqdm
import os
import requests
from io import StringIO
import glob
import re
import zipfile
import openpyxl

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Modules
from utils import primary_dataset, wiki_scraper, convert_to_decimal_degrees

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -v
%watermark --iversions

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 9.9.0

geopandas : 1.1.2
matplotlib: 3.10.8
numpy     : 2.4.1
openpyxl  : 3.1.5
pandas    : 3.0.0
plotly    : 6.5.2
re        : 2.2.1
requests  : 2.32.5
scipy     : 1.17.0
seaborn   : 0.13.2
sklearn   : 1.8.0
tqdm      : 4.67.1



## Load Data

#### Primary Dataset

In [2]:
# Water Quality Data: we will start out with a primary dataset, and follow up by joining other datasets to add variability.

wq_data = primary_dataset.primary_dataset()
print('Initial shape:',wq_data.shape)
wq_data.head()

We will explore Water Quality over the course of 60 months.
Initial shape: (9319, 14)


,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,pet,total alkalinity,electrical conductance,dissolved reactive phosphorus,month
0,-28.760833,17.730278,2011-01-02,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0,2011-01-31
1,-26.861111,28.884722,2011-01-03,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0,2011-01-31
2,-26.450000,28.085833,2011-01-03,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0,2011-01-31
3,-27.671111,27.236944,2011-01-03,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0,2011-01-31
4,-27.356667,27.286389,2011-01-03,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0,2011-01-31


#### Secondary Dataset 1.0 - List of rivers in South Africa

- Wouldn't be nice to join some data directly from the internet, without crowding space on the local machine?
1. Bing Search: "how to read a table from wikipedia pandas" --> ```HTTPError: HTTP Error 403: Forbidden```
2. Bing Search: "HTTPError: HTTP Error 403: Forbidden" --> https://stackoverflow.com/questions/16627227/how-do-i-avoid-http-error-403-when-web-scraping-with-python
3. Add ```import requests```, ```from io import StringIO```, *header* and ```response = requests.get(url, headers=headers)``` PLUS ``StringIO(response.text)`` --> https://docs.python-requests.org/ & https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html

In [3]:
# Scrape a list of rivers from South Africa @ https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa
# Custom headers to mimic a real browser
# def wiki_scraper(url="https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa", headers={"User-Agent": "Chrome/107.0.0.0 Safari/537.36"}, table=1, subset='Mouth / junction coordinates'):

rivers = wiki_scraper.wiki_scraper(url='https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa')

# Convert Degrees, Minutes and Seconds to Decimal Degrees
# Formula: Decimal Degrees = Degrees + (Minutes ÷ 60) + (Seconds ÷ 3,600)
# For example, to convert 30° 15′ 50″: Decimal Degrees = 30 + (15 ÷ 60) + (50 ÷ 3,600) = 30.2639°.
# https://stackoverflow.com/questions/33997361/how-to-convert-degree-minute-second-to-degree-decimal

# extensive 'print' debugging to figure out '\ufeff' was at the end of each string in the pd.Series
rivers['mouth/junctioncoordinates'] = rivers['mouth/junctioncoordinates'].apply(lambda x: x.replace('\ufeff', ""))
rivers['latitude'] = convert_to_decimal_degrees.convert_to_decimal_degrees(rivers['mouth/junctioncoordinates'])['latitude']
rivers['longitude'] = convert_to_decimal_degrees.convert_to_decimal_degrees(rivers['mouth/junctioncoordinates'])['longitude']
rivers= rivers[rivers['latitude'].apply(lambda x: isinstance(x, float))]

# round to 5 decimals
rivers['latitude'] = round(rivers['latitude'].astype(float), 5)
rivers['longitude'] = round(rivers['longitude'].astype(float), 5)

# all river names lower case
rivers['river'] = rivers['river'].str.lower()


rivers.info()

<class 'pandas.DataFrame'>
Index: 50 entries, 0 to 274
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   river                           50 non-null     str    
 1   drainagebasin[a]                29 non-null     str    
 2   provinceandlocation             50 non-null     str    
 3   sourcelocation(town/mountains)  36 non-null     str    
 4   tributaryof(river)              25 non-null     str    
 5   daminriver                      19 non-null     str    
 6   mouth/junctionatlocation(town)  43 non-null     str    
 7   mouth/junctioncoordinates       50 non-null     str    
 8   latitude                        50 non-null     float64
 9   longitude                       50 non-null     float64
dtypes: float64(2), str(8)
memory usage: 4.3 KB


#### Secondary Dataset 1.1 - River Mouths

Using the rivers website above, we separated the river that end in a junction from rivers that end at a mouth. The two terms can be used interchangeably, however mouth refers to a river ending in a ocean/sea or lake, while a junction refers to a river running into another river. This nuance should be noted befor exploration. 

In [4]:
river_mouths = pd.read_excel('data/south_africa_river_mouths.xlsx')
river_mouths['latitude'] = round(river_mouths['latitude'], 5)
river_mouths['longitude'] = round(river_mouths['longitude'], 5)

# all river_names lowercase
river_mouths['river'] = river_mouths['river_name'].str.lower()
river_mouths = river_mouths[['river_name','province','mouth_location','latitude','longitude','river']] ######
river_mouths.info()

<class 'pandas.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   river_name      21 non-null     str    
 1   province        21 non-null     str    
 2   mouth_location  21 non-null     str    
 3   latitude        21 non-null     float64
 4   longitude       21 non-null     float64
 5   river           21 non-null     str    
dtypes: float64(2), str(4)
memory usage: 1.1 KB


#### Secondary Dataset 1.2 - Population Density

In [5]:

path = 'data/Population_density'
if not os.path.exists(path):
    dir_list = os.listdir()
    for file in dir_list:
        # print(file)
        if file.endswith('.zip'):
            with zipfile.ZipFile(file) as f:
                f.extractall(path)
                

# Source - https://stackoverflow.com/q/20906474
# Posted by jonas, modified by community. See post 'Timeline' for change history
# Retrieved 2026-01-30, License - CC BY-SA 4.0

# Map country codes to full names (only South African population)
country_map = {"zaf": "South Africa"}

# Get data file names
filenames = glob.glob(path + "/*.zip") # .csv
# print(filenames)

# Initiate a list of population dataframes
all_dfs = []

for filename in filenames:

    parts = filename.split("_")
    parts = filename.split("\\")
    parts = parts[-1].split("_")
    # print(parts)
    # Basic safety check on filename structure
    if len(parts) < 3:
        continue

    # Extract metadata from filename
    country_code = parts[0].lower()
    year = int(parts[2])
    country_name = country_map.get(country_code, "Unknown")

    # Read CSV
    df = pd.read_csv(filename)

    # Rename XYZ columns to descriptive names
    df = df.rename(columns={
        "X": "longitude",
        "Y": "latitude",
        "Z": "population_density"
    })

    # Insert metadata columns
    df.insert(0, "country", country_name)
    df.insert(1, "year", year)

    all_dfs.append(df)

# Combine all countries & years into one DataFrame
population_density_all = pd.concat(all_dfs, ignore_index=True)

# Sort for consistency
population_density_all = population_density_all.sort_values(
    by=["country", "year", "latitude", "longitude", "population_density"]
).reset_index(drop=True)

# Quick check
population_density_all.head()


,country,year,longitude,latitude,population_density
0,South Africa,2011,37.794583,-46.979583,0.0
1,South Africa,2011,37.802917,-46.979583,0.0
2,South Africa,2011,37.811250,-46.979583,0.0
3,South Africa,2011,37.819583,-46.979583,0.0
4,South Africa,2011,37.827917,-46.979583,0.0


#### Secondary Dataset 1.3 - Countries and Provinces

- 1.3.1 Load and add countries to primary dataset so each sample can be linked to a nominal location

In [6]:
# 1) Copy original dataset
df = wq_data.copy()

# 2) Clean column names (safe + recommended)
df.columns = df.columns.str.strip().str.lower()

# 3) Create GeoDataFrame from lat/lon
gdf_country_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

# 4) Load Natural Earth country boundaries
# https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-details/
countries = gpd.read_file(
    "data/new_earth/ne_50m_admin_0_countries.shp" # other dependent files (.shx, .dhp, etc.) are located in the same directory*
)#.to_crs("EPSG:4326")
# WARNINGS SUPPRESSED

# 5) Spatial join to add country column
gdf_with_country = gpd.sjoin(
    gdf_country_points,
    countries[["ADMIN", "geometry"]],
    how="left",
    predicate="within"
).to_crs("EPSG:4326")

# 6) Rename + cleanup country columns
gdf_with_country = (
    gdf_with_country
    .rename(columns={"ADMIN": "country"})
    .drop(columns=["geometry", "index_right"], errors="ignore")
)

# 7) Move `country` to be the FIRST column
cols = gdf_with_country.columns.tolist()
cols.insert(0, cols.pop(cols.index("country")))
gdf_with_country = gdf_with_country[cols]

# 8) Quick check
gdf_with_country.head()

,country,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,pet,total alkalinity,electrical conductance,dissolved reactive phosphorus,month
0,Namibia,-28.760833,17.730278,2011-01-02,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0,2011-01-31
1,South Africa,-26.861111,28.884722,2011-01-03,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0,2011-01-31
2,South Africa,-26.450000,28.085833,2011-01-03,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0,2011-01-31
3,South Africa,-27.671111,27.236944,2011-01-03,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0,2011-01-31
4,South Africa,-27.356667,27.286389,2011-01-03,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0,2011-01-31


- 1.3.2 Load and add provinces too

In [7]:
# 1) Copy dataset
df = gdf_with_country.copy()

# 2) Clean column names (safe + recommended)
df.columns = df.columns.str.strip().str.lower()

# 3) Create GeoDataFrame from lat/lon
gdf_prov_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

# 4) Load Natural Earth country boundaries
# https://simplemaps.com/gis/country/za
path_to_data = 'data/za_shp' # path to downloaded shapefile
provinces = gpd.read_file(path_to_data, layer='za')

# 5) Spatial join to add country column
# province column
gdf_with_province = gpd.sjoin(
    gdf_prov_points,
    provinces[["name", "geometry"]],
    how="left",
    predicate="within"
).to_crs("EPSG:4326")

# 6) Rename + cleanup country and province columns
gdf_with_province = (
    gdf_with_province
    .rename(columns={"name": "province"})
    .drop(columns=["id", "geometry", "index_right"], errors="ignore")
)

# 7) Finalize added column operation

final_df = gdf_with_province.copy()

# 8) Quick check
final_df.head()

,country,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,pet,total alkalinity,electrical conductance,dissolved reactive phosphorus,month,province
0,Namibia,-28.760833,17.730278,2011-01-02,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0,2011-01-31,Northern Cape
1,South Africa,-26.861111,28.884722,2011-01-03,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0,2011-01-31,Mpumalanga
2,South Africa,-26.450000,28.085833,2011-01-03,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0,2011-01-31,Gauteng
3,South Africa,-27.671111,27.236944,2011-01-03,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0,2011-01-31,Free State
4,South Africa,-27.356667,27.286389,2011-01-03,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0,2011-01-31,Free State


In [8]:
# There actually happens to be other countries in the mix, but we only want to observe South African Rivers
final_df = final_df[final_df['country']=='South Africa']

#### Now we can join the population density variable to the dataset

In [9]:

# --- 1) Copies + normalize column names ---
wq = final_df.copy()
pd_all = population_density_all.copy()

wq.columns = wq.columns.str.strip().str.lower()
pd_all.columns = pd_all.columns.str.strip().str.lower()

# --- 2) Ensure numeric types ---
for col in ["longitude", "latitude"]:
    wq[col] = pd.to_numeric(wq[col], errors="coerce")
    pd_all[col] = pd.to_numeric(pd_all[col], errors="coerce")

pd_all["population_density"] = pd.to_numeric(pd_all["population_density"], errors="coerce")
pd_all["year"] = pd.to_numeric(pd_all["year"], errors="coerce")

# --- 3) Extract sample year ---
wq["sample_year"] = pd.to_datetime(wq["sample date"], errors="coerce").dt.year

# --- 4) Initialize output columns ---
wq["pd_year"] = np.nan
wq["pop_density_nn"] = np.nan
wq["distance_km_to_pd_cell"] = np.nan

# --- 5) Helper: approx distance in km ---
def approx_deg_to_km(dlon, dlat, lat):
    km_lat = dlat * 111.32
    km_lon = dlon * 111.0 * np.cos(np.deg2rad(lat))
    return np.sqrt(km_lat**2 + km_lon**2)

# --- 6) Filter PD to clean rows only ---
pd_all_clean = pd_all.dropna(subset=["year", "longitude", "latitude", "population_density"]).copy()
pd_all_clean["year"] = pd_all_clean["year"].astype(int)

# --- 7) Build KDTree per year once ---
trees = {}
pd_by_year = {}

for yr, group in pd_all_clean.groupby("year"):
    group = group.reset_index(drop=True)
    coords = group[["longitude", "latitude"]].to_numpy()
    trees[yr] = cKDTree(coords)
    # print(coords)
    pd_by_year[yr] = group

# --- 8) Match WQ points year-by-year ---
valid_wq = wq.dropna(subset=["sample_year", "longitude", "latitude"]).copy()
valid_wq["sample_year"] = valid_wq["sample_year"].astype(int)

for yr in sorted(valid_wq["sample_year"].unique()):
    if yr not in trees:
        print(yr,'not in trees')
        continue

    idx_rows = valid_wq.index[valid_wq["sample_year"] == yr]
    query_pts = valid_wq.loc[idx_rows, ["longitude", "latitude"]].to_numpy()

    dist, idx = trees[yr].query(query_pts, k=1)
    matched = pd_by_year[yr].iloc[idx].reset_index(drop=True)

    wq.loc[idx_rows, "pd_year"] = yr
    wq.loc[idx_rows, "pop_density_nn"] = matched["population_density"].to_numpy()

    dlon = valid_wq.loc[idx_rows, "longitude"].to_numpy() - matched["longitude"].to_numpy()
    dlat = valid_wq.loc[idx_rows, "latitude"].to_numpy() - matched["latitude"].to_numpy()
    lat  = valid_wq.loc[idx_rows, "latitude"].to_numpy()

    wq.loc[idx_rows, "distance_km_to_pd_cell"] = approx_deg_to_km(dlon, dlat, lat)

# --- 9) Build FINAL joined dataset: all original columns + new features ---
original_cols = final_df.copy()
original_cols.columns = original_cols.columns.str.strip().str.lower()
original_cols = original_cols.columns.tolist()

new_cols = ["sample_year", "pop_density_nn", "distance_km_to_pd_cell"]

# Keep original order + add new cols at the end (only if they exist)
final_cols = original_cols + [c for c in new_cols if c in wq.columns]
wq_joined = wq[final_cols].copy()

# --- 10) Take a peak ---
wq_joined.head()

,country,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,pet,total alkalinity,electrical conductance,dissolved reactive phosphorus,month,province,sample_year,pop_density_nn,distance_km_to_pd_cell
1,South Africa,-26.861111,28.884722,2011-01-03,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0,2011-01-31,Mpumalanga,2011,5.049022,0.251555
2,South Africa,-26.450000,28.085833,2011-01-03,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0,2011-01-31,Gauteng,2011,23.239988,0.419537
3,South Africa,-27.671111,27.236944,2011-01-03,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0,2011-01-31,Free State,2011,687.465759,0.069958
4,South Africa,-27.356667,27.286389,2011-01-03,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0,2011-01-31,Free State,2011,6.092811,0.232396
5,South Africa,-27.010111,26.698083,2011-01-04,12433.5,10433.5,9579.5,8531.5,0.129651,0.042672,98.6,82.200,289.8,192.0,2011-01-31,Free State,2011,77.849716,0.466183


In [10]:
wq_joined["distance_km_to_pd_cell"].agg(
    ["min", "max", "mean", "median"]
)

min       0.025893
max       0.783463
mean      0.347419
median    0.364354
Name: distance_km_to_pd_cell, dtype: float64

**How to interpret this**

You’re matching water-quality points to 1 km population-density grid cells.

Your results:
	•	Mean distance ≈ 0.34 km (340 meters)
	•	Median ≈ 0.36 km
	•	Max ≈ 0.61 km

For a 1 km grid, this is exactly what we want to see.

Why?

	•	In a 1 km × 1 km cell, the furthest possible distance from the center is ~0.71 km.
	•	Your max (0.61 km) is below that theoretical worst case.
	•	A mean around 0.35 km means most matches are happening well within the same grid cell or very close neighbors.

This tells us:

	•	Spatial alignment is tight
	•	Nearest-neighbor matching worked correctly
	•	No red flags like multi-km mismatches

## Lastly Join River Names and Junction Coordinates

```rivers```  
```river_mouths```

In [11]:
# distinguish river mouths from junctions
river_junctions = rivers[~rivers['river'].isin(river_mouths['river'])]
river_junctions.head()

,river,drainagebasin[a],provinceandlocation,sourcelocation(town/mountains),tributaryof(river),daminriver,mouth/junctionatlocation(town),mouth/junctioncoordinates,latitude,longitude
0,amandawe river,NaN,"Amandawe, KwaZulu-Natal",north west of Amandawe,north of Scottburgh,NaN,Umpambanyoni River,30°15′30″S 30°45′0″E / 30.25833°S 30.75000°E,-30.25833,30.75000
1,amahlongwa river,NaN,"Amahlongwa, KwaZulu-Natal",south west of Amahlongwa,west of uMkomaas,NaN,Indian Ocean,30°15′30″S 30°44′0″E / 30.25833°S 30.73333°E,-30.25833,30.73333
3,apies river,A2,"Gauteng, Tshwane, Pretoria","Bronberg, southeast of Pretoria","Moretele River, then Crocodile River and Limpo...",Bon Accord Dam,Makapanstad,25°14′24″S 28°08′36″E / 25.24000°S 28.14333°E,-25.24000,28.14333
4,as river (or axel river),C8,Free State,Southeast of Bethlehem,"Liebenbergsvlei River, then Wilge River",Sol Plaatjie Dam,NaN,28°13′27″S 28°21′58″E / 28.22417°S 28.36611°E,-28.22417,28.36611
5,assegaai river,W5,Mpumalanga,North of Wakkerstroom,Mkondo River,Heyshope Dam,Swaziland border,27°04′46″S 31°02′19″E / 27.07944°S 31.03861°E,-27.07944,31.03861


**NOTE: River mouth and river junction cannot be used interchangeably.**

- River Mouth: where a river ends in a body of water, such as a lake, sea or ocean.
- River Junction: where a river ends by intersecting, or running into another river.

Since this dataset also offers geographical coordinates, we can leverage that information to find nearest natural landmarks. We need to make a large world decision here. In conclusion, we decided any sample locations within 10 km of a river mouth / junction should be representative of the water quality of that river mouth / junction.

How many sample locations are near a river mouth / junction?

In [12]:
def river_mouth(THRESH_KM: int) -> pd.DataFrame:
    '''
    insert description
    '''
    THRESH_KM = THRESH_KM
    EARTH_RADIUS_KM = 6371

    # river mouth coordinates
    mouth_coords = np.radians(
        river_mouths[["latitude", "longitude"]].astype(float).dropna().to_numpy()
    )

    tree = BallTree(mouth_coords, metric="haversine")

    # sampling point coordinates
    sample_coords = np.radians(
        wq_joined[["latitude", "longitude"]].to_numpy()
    )

    dist_rad, _ = tree.query(sample_coords, k=1)

    # single binary flag (1 = river mouth, 0 = not)
    wq_joined["river_mouth"] = (
        dist_rad[:, 0] * EARTH_RADIUS_KM <= THRESH_KM
    ).astype(int)

    return wq_joined


In [13]:
def river_junction(THRESH_KM: int) -> pd.DataFrame:
    '''
    insert description
    '''
    THRESH_KM = THRESH_KM
    EARTH_RADIUS_KM = 6371

    # river mouth coordinates
    mouth_coords = np.radians(
        river_junctions[["latitude", "longitude"]].astype(float).dropna().to_numpy() # toggle river df
    )

    tree = BallTree(mouth_coords, metric="haversine")

    # sampling point coordinates
    sample_coords = np.radians(
        wq_joined[["latitude", "longitude"]].to_numpy()
    )

    dist_rad, _ = tree.query(sample_coords, k=1)

    # single binary flag (1 = river mouth, 0 = not)
    wq_joined["river_junction"] = (
        dist_rad[:, 0] * EARTH_RADIUS_KM <= THRESH_KM
    ).astype(int)

    return wq_joined

In [14]:
def river_mouthORjunction(THRESH_KM: int) -> pd.DataFrame:
    '''
    insert description
    '''
    THRESH_KM = THRESH_KM
    EARTH_RADIUS_KM = 6371

    # river mouth coordinates
    mouth_coords = np.radians(
        rivers[["latitude", "longitude"]].astype(float).dropna().to_numpy() # toggle river df
    )

    tree = BallTree(mouth_coords, metric="haversine")

    # sampling point coordinates
    sample_coords = np.radians(
        wq_joined[["latitude", "longitude"]].to_numpy()
    )

    dist_rad, _ = tree.query(sample_coords, k=1)

    # single binary flag (1 = river mouth, 0 = not)
    wq_joined["river_mouthORjunction"] = (
        dist_rad[:, 0] * EARTH_RADIUS_KM <= THRESH_KM
    ).astype(int)

    return wq_joined

In [15]:
wq_joined= river_mouth(10)
wq_joined = river_junction(10)
wq_joined = river_mouthORjunction(10)

In [16]:

thresh_dict = {}
for i in range(12):
    try:
        thresh_dict[i+1] = river_mouth(i+1)["river_mouth"].value_counts()[1]
    except:
        thresh_dict[i+1] = 0

        
listzipped = list(zip(thresh_dict.keys(), thresh_dict.values()))

pd.DataFrame(listzipped, index=thresh_dict.keys(), columns=['Threshold (kilometers)', 'Number of samples near river mouths']).set_index('Threshold (kilometers)')

,Number of samples near river mouths
Threshold (kilometers),
1,0
2,0
3,0
4,0
5,0
6,0
7,51
8,51
9,51


Logic: As you can see in the table above, the larger the distance threshold, the more sample locations are "near a river mouth". We want to exclude a reasonable amount of sample locations while also maintaining a healthy amount of data to compare with to represent water samples located "near a river mouth".

### Handling Missing Values

It's important for us to know how many attributes we are working with. Below, we can see that there are 19 columns of data to choose from so far, and 9155 rows for each.

Missing values in the dataset were carefully handled to ensure data consistency and granularity. 

In [17]:
# eloquently sort values while observing if there are any missing values
# wq_joined = river_mouth(THRESH_KM=10)#.reindex()
wq_joined = wq_joined.dropna(subset='province')
print('shape:',wq_joined.shape)
wq_joined.to_csv('data/wq.csv', index=False)
wq_joined.isna().sum().sort_values(ascending=False)

shape: (9093, 22)


country                          0
latitude                         0
longitude                        0
sample date                      0
nir                              0
green                            0
swir16                           0
swir22                           0
ndmi                             0
mndwi                            0
pet                              0
total alkalinity                 0
electrical conductance           0
dissolved reactive phosphorus    0
month                            0
province                         0
sample_year                      0
pop_density_nn                   0
distance_km_to_pd_cell           0
river_mouth                      0
river_junction                   0
river_mouthORjunction            0
dtype: int64

'NA' was dropped where subset='province' because there were only ~100 'NA' out of over 9000 datapoints leftover. We have a clean dataset without missing values at our disposal for further exploration.

In [18]:
wq_joined.tail()

,country,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,...,electrical conductance,dissolved reactive phosphorus,month,province,sample_year,pop_density_nn,distance_km_to_pd_cell,river_mouth,river_junction,river_mouthORjunction
9314,South Africa,-27.527500,30.858056,2015-12-23,15296.5,10043.0,16381.0,14443.0,-0.034236,-0.239858,...,134.0,20.0,2015-12-31,KwaZulu-Natal,2015,79.402359,0.390648,0,0,0
9315,South Africa,-26.861111,28.884722,2015-12-23,15642.5,10294.5,17045.5,14710.0,-0.042921,-0.246928,...,388.0,20.0,2015-12-31,Mpumalanga,2015,15.326218,0.251555,0,0,0
9316,South Africa,-26.984722,26.632278,2015-12-23,14945.0,10732.0,18303.0,16281.0,-0.100999,-0.260754,...,835.0,148.0,2015-12-31,North West,2015,7.830057,0.529907,0,0,0
9317,South Africa,-27.935000,26.126667,2015-12-23,14727.5,11051.0,18420.0,15724.5,-0.111396,-0.250042,...,305.0,28.0,2015-12-31,Free State,2015,3.002162,0.347003,0,0,0
9318,South Africa,-29.744167,29.905833,2015-12-31,15344.5,10461.5,14948.5,12747.0,0.013072,-0.176584,...,86.4,5.0,2015-12-31,KwaZulu-Natal,2015,107.789825,0.364354,0,0,0


In [19]:
# raise NotImplementedError()